# Estimating COVID-19's $R_t$ in Real-Time with PYMC3

Fork of Kevin Systrom's notebook by Grzegorz Kossakowski & Maciek Zdanowicz that plugs the EU data instead of the US one as the input.

Model originally built by [Thomas Vladeck](https://github.com/tvladeck) in Stan, parts inspired by the work over at https://epiforecasts.io/, lots of help from [Thomas Wiecki](https://twitter.com/twiecki). Thank you to everyone who helped.

This notebook is a WIP - I'll add more context and commentary over the coming week.

In [3]:
# For some reason Theano is unhappy when I run the GP, need to disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import requests
import pymc3 as pm
import pandas as pd
import numpy as np
import theano
import theano.tensor as tt

from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib import ticker

from datetime import date
from datetime import datetime
from datetime import timedelta

from IPython.display import clear_output

%config InlineBackend.figure_format = 'retina'

## Neverland

In [38]:
dates_considered = pd.date_range(end=pd.to_datetime("today"),start='15-02-2020')

In [11]:
Rt = np.array([2.5]*10 + [2.2]*10 + [1.9]*10 + [1.6]*5 + [1.3]*5 + [1.15]*5 + [1.0]*50)
serial_intervalConst = 5.0
Rt += np.random.normal(0,0.1,len(Rt)) # (R_t - 1)/serial_interval = theta
Theta = (Rt - 1)/serial_intervalConst
cases_neverland = pd.Series(data=(10*np.exp(Theta.cumsum())).round(),index=pd.date_range(end=pd.to_datetime("today"),periods=len(Rt)),name='cases')

In [12]:
cases_neverland

2020-02-11 13:24:19.434850       14.0
2020-02-12 13:24:19.434850       19.0
2020-02-13 13:24:19.434850       26.0
2020-02-14 13:24:19.434850       34.0
2020-02-15 13:24:19.434850       46.0
                               ...   
2020-05-11 13:24:19.434850    41339.0
2020-05-12 13:24:19.434850    41983.0
2020-05-13 13:24:19.434850    42143.0
2020-05-14 13:24:19.434850    40979.0
2020-05-15 13:24:19.434850    42503.0
Freq: D, Name: cases, Length: 95, dtype: float64

In [13]:
cases_neverland.plot()

In [35]:
p_delay = pd.read_csv('data/pdelay.csv',index_col=['index'])['pdelay']

### Translate Confirmation Dates to Onset Dates

Our goal is to translate positive test counts to the dates where they likely occured. Since we have the distribution, we can distribute case counts back in time according to that distribution. To accomplish this, we reverse the case time series, and convolve it using the distribution of delay from onset to confirmation. Then we reverse the series again to obtain the onset curve. Note that this means the data will be 'right censored' which means there are onset cases that have yet to be reported so it looks as if the count has gone down.

In [40]:
# Maps pd.Series to pd.Series indexed by dates_considered
def confirmed_to_onset(confirmed, p_delay):
    assert not confirmed.isna().any()
    # First, we take a convolution with pdelay to distribute the cases confirmed at a certain time
    # to the appropriate date of onset 
    convolved = np.convolve(confirmed[::-1].values, p_delay)
    # Since covolution increases the size of the array, we clip the data in order to to fit the index
    return pd.Series(np.flip(convolved[:dates_considered.size]), index=dates_considered, name='Onset')

In [41]:
onset = confirmed_to_onset(cases_neverland, p_delay)

In [42]:
onset.plot()

### Adjust for Right-Censoring

Since we distributed observed cases into the past to recreate the onset curve, we now have a right-censored time series. We can correct for that by asking what % of people have a delay less than or equal to the time between the day in question and the current day.

For example, 5 days ago, there might have been 100 cases onset. Over the course of the next 5 days some portion of those cases will be reported. This portion is equal to the cumulative distribution function of our delay distribution. If we know that portion is say, 60%, then our current count of onset on that day represents 60% of the total. This implies that the total is 166% higher. We apply this correction to get an idea of what actual onset cases are likely, thus removing the right censoring.

In [43]:
def adjust_onset_for_right_censorship(onset, p_delay):
    cumulative_p_delay = p_delay.cumsum()
    # Calculate the additional ones needed so shapes match
    ones_needed = np.max([len(onset) - len(cumulative_p_delay),0])
    padding_shape = (0, ones_needed)
    # Add ones and flip back
    cumulative_p_delay = np.pad(
        cumulative_p_delay,
        padding_shape,
        constant_values=1)
    cumulative_p_delay = np.flip(cumulative_p_delay[:onset.size])
    # Adjusts observed onset values to expected terminal onset values
    adjusted = onset / cumulative_p_delay
    
    return adjusted, cumulative_p_delay

In [44]:
adjusted, cumulative_p_delay = adjust_onset_for_right_censorship(onset, p_delay)

Take a look at all three series: confirmed, onset and onset adjusted for right censoring.

In [45]:
fig, ax = plt.subplots(figsize=(5,3))
country = "Neverland"

cases_neverland.plot(
    ax=ax,
    label='Confirmed',
    title=country,
    c='k',
    alpha=.25,
    lw=1)

onset.plot(
    ax=ax,
    label='Onset',
    c='k',
    lw=1)

adjusted.plot(
    ax=ax,
    label='Adjusted Onset',
    c='k',
    linestyle='--',
    lw=1)

ax.legend();

Let's have the model run on days where we have enough data ~last 50 or so

### Sample the Posterior with PyMC3

We assume a poisson likelihood function and feed it what we believe is the onset curve based on reported data. We model this onset curve based on the same math in the previous notebook:

$$ I^\prime = Ie^{\gamma(R_t-1)} $$

We define $\theta = \gamma(R_t-1)$ and model $ I^\prime = Ie^{\theta} $ where $\theta$ observes a random walk. We let $\gamma$ vary independently based on known parameters for the serial interval. Therefore, we can recover $R_t$ easily by $R_t = \frac{\theta}{\gamma}+1$

The only tricky part is understanding that we're feeding in _onset_ cases to the likelihood. So $\mu$ of the poisson is the positive, non-zero, expected onset cases we think we'd see today.

We calculate this by figuring out how many cases we'd expect there to be yesterday total when adjusted for bias and plugging it into the first equation above. We then have to re-bias this number back down to get the expected amount of onset cases observed that day.

In [47]:
class MCMCModel(object):
    
    def __init__(self, region, onset, cumulative_p_delay, window=50):
        
        # Just for identification purposes
        self.region = region
        
        # For the model, we'll only look at the last N
        self.onset = onset.iloc[-window:]
        self.data = onset/cumulative_p_delay
        self.cumulative_p_delay = cumulative_p_delay[-window:]
        
        # Where we store the results
        self.trace = None
        self.trace_index = self.onset.index[1:]

    def run(self, chains=1, tune=3000, draws=1000, target_accept=.95):

        with pm.Model() as model:

            # Random walk magnitude
            step_size = pm.HalfNormal('step_size', sigma=.1)

            # Theta random walk
            theta_raw_init = pm.Normal('theta_raw_init', 0.3, 0.2)
            theta_raw_steps = pm.Laplace('theta_raw_steps', mu=0, b=step_size, shape=len(self.onset)-2)
            theta_raw = tt.concatenate([[theta_raw_init], theta_raw_steps])
            theta = pm.Deterministic('theta', theta_raw.cumsum())

            # Let the serial interval be a random variable and calculate r_t
            serial_interval = pm.Gamma('serial_interval', alpha=200.0, beta=40.0)
            r_t = pm.Deterministic('r_t', theta*serial_interval + 1)

            # Trying old version now
            inferred_yesterday = self.onset.values[:-1] / self.cumulative_p_delay[:-1] 
            expected_today = inferred_yesterday * pm.math.exp(theta) * (self.cumulative_p_delay[1:]) 

            #inferred_yesterday = self.data.values[:-1]
            #expected_today = inferred_yesterday * pm.math.exp(theta) 

            # Ensure cases stay above zero for poisson
            mu = pm.math.maximum(.1, expected_today)
            # observed = self.data.values[1:]
            observed = self.onset.round().values[1:]
            cases = pm.Poisson('cases', mu=mu, observed=observed)

            self.trace = pm.sample(
                chains=chains,
                tune=tune,
                draws=draws,
                target_accept=target_accept)
            
            return self

### Run Pymc3 Model

In [46]:
def df_from_model(model):
    
    r_t = model.trace['r_t']
    serial_interval = model.trace['serial_interval']
    theta_t = model.trace['theta']
    
    mean = np.mean(r_t, axis=0)
    median = np.median(r_t, axis=0)
    hpd_90 = pm.stats.hpd(r_t, credible_interval=.9)
    hpd_50 = pm.stats.hpd(r_t, credible_interval=.5)
    
    mean_theta = np.mean(theta_t, axis=0)
    median_theta = np.median(theta_t, axis=0)
    hpd_90_theta = pm.stats.hpd(theta_t, credible_interval=.9)
    hpd_50_theta = pm.stats.hpd(theta_t, credible_interval=.5)
    
    
    idx = pd.MultiIndex.from_product([
            [model.region],
            model.trace_index
        ], names=['region', 'date'])
        
    df = pd.DataFrame(data=np.c_[mean, median, hpd_90, hpd_50], index=idx,
                 columns=['mean', 'median', 'lower_90', 'upper_90', 'lower_50','upper_50'])
    dfTheta = pd.DataFrame(data=np.c_[mean_theta, median_theta, hpd_90_theta, hpd_50_theta], index=idx,
                 columns=['mean', 'median', 'lower_90', 'upper_90', 'lower_50','upper_50'])
    return df, dfTheta, serial_interval

def create_and_run_model(name, country):
    confirmed = country.cases
    onset = confirmed_to_onset(confirmed, p_delay)
    _ , cumulative_p_delay = adjust_onset_for_right_censorship(onset, p_delay)
    return MCMCModel(name, onset, cumulative_p_delay, window=1000).run(tune=1000)

In [48]:
model_neverland = create_and_run_model('Neverland', pd.DataFrame(cases_neverland))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [serial_interval, theta_raw_steps, theta_raw_init, step_size]
Sampling chain 0, 0 divergences: 100%|██████████| 2000/2000 [03:37<00:00,  9.20it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


In [57]:
result_neverland, result_neverlandTheta, serial_interval = df_from_model(model_neverland)

### Render Charts

In [50]:
def plot_rt(name, result, ax, c=(.3,.3,.3,1), ci=(0,0,0,.05)):
    ax.set_ylim(0.0, 2)
    ax.set_title(name)
    ax.plot(result['median'],
            marker='o',
            markersize=4,
            markerfacecolor='w',
            lw=1,
            c=c,
            markevery=2)
    ax.fill_between(
        result.index,
        result['lower_90'].values,
        result['upper_90'].values,
        color=ci,
        lw=0)
    ax.axhline(1.0, linestyle=':', lw=1)
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))

In [51]:
def plot_Thetat(name, result, ax, c=(.3,.3,.3,1), ci=(0,0,0,.05)):
    ax.set_ylim(-0.05, 0.35)
    ax.set_title(name)
    ax.plot(result['median'],
            marker='o',
            markersize=4,
            markerfacecolor='w',
            lw=1,
            c=c,
            markevery=2)
    ax.fill_between(
        result.index,
        result['lower_90'].values,
        result['upper_90'].values,
        color=ci,
        lw=0)
    ax.axhline(1.0, linestyle=':', lw=1)
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    ax.xaxis.set_major_locator(mdates.WeekdayLocator(interval=2))

In [60]:
fig, axes = plt.subplots(
    nrows=2,
    ncols=1,
figsize=(8,8))

# for ax, (country, result) in zip(axes.flat, results.groupby('region')):
#     plot_rt(country, result.droplevel(0), ax)
plot_rt('Neverland', result_neverland.droplevel(0), axes[0])
ax = axes[0].twinx()  # instantiate a second axes that shares the same x-axis
ax.plot(onset)
ax.plot(adjusted)

print(f"Serial interval: {serial_intervalConst}\nRt: {Rt}")

dataTheta = result_neverlandTheta.droplevel(0)
plot_Thetat('Neverland', dataTheta, axes[1])
index = dataTheta.index
thetaNever = ((Rt-1)/serial_intervalConst)[:index.size]
thetaNeverSeries = pd.Series(data=thetaNever,index=dataTheta.index[:index.size])
axes[1].plot(thetaNeverSeries)

fig.tight_layout()
fig.set_facecolor('w')

Serial interval: 5.0
Rt: [2.6720245  2.44043367 2.66360943 2.36593843 2.46715959 2.48665785
 2.47373213 2.45183005 2.5271453  2.45687601 2.17078311 2.09465708
 2.2285025  2.44527871 2.24310614 2.11147787 2.1060709  2.19631979
 2.1471289  2.12995793 1.86053127 1.87046365 1.9148266  1.92479571
 1.96236075 1.93238268 1.90971465 1.84782881 1.96089582 1.77571763
 1.66451617 1.54281968 1.50466008 1.58182609 1.65920284 1.17102273
 1.32386556 1.09999169 1.12939857 1.45256103 1.04522992 1.15986558
 1.11312391 1.08076057 1.2601864  1.09444578 1.07881312 0.90063685
 1.17278252 1.11401371 0.92761095 0.88932083 1.07223315 0.98154112
 0.97511937 0.98118108 1.13483788 1.1572882  1.00539245 0.99783869
 1.01258485 1.02259793 0.92429643 1.0100114  1.02187695 0.97734123
 0.84895982 1.01864387 1.14362093 1.10588671 0.9721851  1.06898375
 1.12649308 1.12186132 1.01484145 0.97977799 1.11750684 1.08370248
 0.85387029 1.0067645  0.84898456 1.02391532 1.06937157 1.01776757
 0.99881168 1.15447176 1.01504368 0.9

NameError: name 'dataTheta' is not defined

In [58]:
plt.hist(serial_int,bins=50)
print(np.mean(serial_interval),np.std(serial_interval))

5.011556422356028 0.35364622067953516
